In [43]:
import pandas as pd

# Load the dataset CSV
df = pd.read_csv("/content/drive/MyDrive/Resume.csv")

# Create a new DataFrame for Hugging Face format
huggingface_df = pd.DataFrame(columns=['text', 'label'])

# Map category names to shorter labels
category_map = {
    'HR': 'HR',
    'DESIGNER': 'Designer',
    'INFORMATION-TECHNOLOGY': 'IT',
    'TEACHER': 'Teacher',
    'ADVOCATE': 'Advocate',
    'BUSINESS-DEVELOPMENT': 'BizDev',
    'HEALTHCARE': 'Healthcare',
    'FITNESS': 'Fitness',
    'AGRICULTURE': 'Agriculture',
    'BPO': 'BPO',
    'SALES': 'Sales',
    'CONSULTANT': 'Consultant',
    'DIGITAL-MEDIA': 'Media',
    'AUTOMOBILE': 'Automobile',
    'CHEF': 'Chef',
    'FINANCE': 'Finance',
    'APPAREL': 'Apparel',
    'ENGINEERING': 'Engineering',
    'ACCOUNTANT': 'Accountant',
    'CONSTRUCTION': 'Construction',
    'PUBLIC-RELATIONS': 'PR',
    'BANKING': 'Banking',
    'ARTS': 'Arts',
    'AVIATION': 'Aviation'
}

# Iterate over each row and append to the new DataFrame
for index, row in df.iterrows():
    text = row['Resume_str']
    category = row['Category']

    # Convert category to uppercase for mapping
    category_upper = category.upper()

    # Map category to shorter label
    label = category_map.get(category_upper, 'Other')  # Default to 'Other' if category not found

    huggingface_df = huggingface_df.append({'text': text, 'label': label}, ignore_index=True)

# Save the new DataFrame to CSV
huggingface_df.to_csv('resume_dataset_huggingface.csv', index=False)


<ipython-input-43-b835ee045638>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  huggingface_df = huggingface_df.append({'text': text, 'label': label}, ignore_index=True)
<ipython-input-43-b835ee045638>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  huggingface_df = huggingface_df.append({'text': text, 'label': label}, ignore_index=True)
<ipython-input-43-b835ee045638>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  huggingface_df = huggingface_df.append({'text': text, 'label': label}, ignore_index=True)
<ipython-input-43-b835ee045638>:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  huggingface_df = huggingface_df.append({'text'

In [44]:
#!pip install huggingface_hub


In [45]:
#!huggingface-cli login


In [46]:
pip install transformers datasets


In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer, DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset
from datasets import load_dataset

In [48]:
# Load the Hugging Face dataset and model
model_name = "distilbert-base-uncased"
dataset = load_dataset("Saba06huggingface/Resume_classification_dataset",split="train")
# Load the Hugging Face dataset
#dataset = Dataset.from_hf("Saba06huggingface/Resume_classification_dataset", split="train")

In [49]:
dataset = pd.DataFrame(dataset)

In [50]:
# Replace 'column_name' with the actual column name containing the resume text
dataset.rename(columns = {'text':'resume'}, inplace = True)

In [51]:
train_dataset, eval_dataset = train_test_split(dataset, test_size=0.2)

In [52]:
train_dataset

,resume,label
2454,PASSENGER SERVICE CONCOURSE MANAGER ...,Aviation
1841,ACCOUNTANT Summary Resul...,Accountant
736,"SLEEP TECHNOLOGIST, RPSGT Summa...",Healthcare
257,INFORMATION TECHNOLOGY CONSULTANT ...,IT
212,"SALES CONSULTANT, INTERIOR DESIGNER...",Designer
...,...,...
1134,CONSULTANT Summary USE MY SK...,Consultant
314,IT MANAGEMENT Career Overvi...,IT
1799,ENGINEERING OPERATIONS DIRECTOR ...,Engineering
327,FREELANCE IT CONSULTANT Career ...,IT


In [53]:
# Load the fine-tuning dataset from Hugging Face Hub
#train_dataset = load_dataset("Saba06huggingface/Resume_classification_dataset", split="train")
#val_dataset = load_dataset("Saba06huggingface/Resume_classification_dataset", split="validation")

In [54]:
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=24)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# Define a function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['resume'], padding="max_length", truncation=True)

In [56]:
train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)

In [63]:
# Define a function to preprocess the labels
def preprocess_function(examples):
    return {'labels': examples['label']}

In [64]:
# Tokenize the datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1987 [00:00<?, ? examples/s]

Map:   0%|          | 0/497 [00:00<?, ? examples/s]

In [70]:
from sklearn.preprocessing import LabelEncoder

In [72]:
# Convert Dataset to pandas DataFrame
train_df = tokenized_train.to_pandas()
eval_df = tokenized_eval.to_pandas()


In [75]:
# Convert labels to numerical format
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
eval_df['label'] = label_encoder.transform(eval_df['label'])


In [80]:
tokenized_train = Dataset.from_pandas(train_df)
tokenized_eval = Dataset.from_pandas(eval_df)


In [81]:
#!pip install --upgrade accelerate
#!pip uninstall -y transformers accelerate
#!pip install transformers accelerate

In [82]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy='steps',
    logging_dir="./logs",
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    load_best_model_at_end=True
)

In [83]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

In [84]:
# Fine-tune the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()

# Print evaluation results
print(results)

Step,Training Loss,Validation Loss
100,2.947100,2.310519
200,1.737400,1.187710
300,1.011300,0.867118
400,0.779300,0.723451
500,0.703600,0.718160
600,0.511700,0.677695
700,0.480300,0.681000


{'eval_loss': 0.7181602716445923, 'eval_runtime': 7.9514, 'eval_samples_per_second': 62.505, 'eval_steps_per_second': 7.923, 'epoch': 3.0}


In [85]:
# Save the model
model.save_pretrained("./resume_classification_model")
tokenizer.save_pretrained("./resume_classification_model")

('./resume_classification_model/tokenizer_config.json',
 './resume_classification_model/special_tokens_map.json',
 './resume_classification_model/vocab.txt',
 './resume_classification_model/added_tokens.json')